In [1]:
!pip install Keras-tuner

     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for Keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=e667cda4f5c66115e9720950b3bf93d9be17724c2db81b63401f242dc0e24b51
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c435d5555abce01e7912ab7b6142ef2ed6552a3c946232211d94c900ef41b45a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built Keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_image, train_lables), (test_image, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_image = train_image/255.0
test_image = test_image/255.0

In [6]:
train_image[0].shape

(28, 28)

In [7]:
train_image = train_image.reshape(len(train_image),28,28,1)
test_image = test_image.reshape(len(test_image),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(filters=hp.Int("conv_1_filter", min_value = 32, max_value = 128, step = 16), 
                                                kernel_size=hp.Choice("conv_1_kernel", values = [3,5]), 
                                                activation='relu', 
                                                input_shape=(28, 28, 1)
                                                ),
                            keras.layers.Conv2D(filters=hp.Int("conv_2_filter", min_value = 32, max_value = 64, step = 16), 
                                                kernel_size=hp.Choice("conv_2_kernel", values = [3,5]), 
                                                activation='relu', 
                                                input_shape=(28, 28, 1)
                                                ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int("dense_1_units", min_value = 32, max_value = 128, step = 16),
                                activation = 'relu'
                            ),
                            keras.layers.Dense(10, activation='softmax') #output_layer                            
                            
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model


In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory = 'output',
                            project_name = "Mnist Fashion"
                            )

In [11]:
tuner_search.search(train_image, train_lables, epochs= 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.9071666598320007

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 00h 03m 19s
INFO:tensorflow:Oracle triggered exit


In [12]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,719,434
Trainable params: 2,719,434
Non-trainable params: 0
_________________________________________________________________
